# 实战：解决BILIBILI的登录问题（一部分）
该实战的目的是实现BILIBILI的手机号登录界面处理验证码的问题。不过发送短信验证码就真的无能为力了。

In [10]:
from selenium.webdriver import Chrome
from selenium.webdriver.common.by import By
web=Chrome()
web.get('https://www.bilibili.com/')

点击登录、点击手机号登录、点击输入框输入手机号码、点击输入验证码

In [11]:
import time
el=web.find_element(By.XPATH,'//*[@id="i_cecream"]/div[2]/div[1]/div[1]/ul[2]/li[1]/li/div[1]/div').click()
time.sleep(1)
lgin=web.find_element(By.XPATH,'/html/body/div[3]/div/div[4]/div[1]/div[3]').click()
time.sleep(1)
phone_num=web.find_element(By.XPATH,'/html/body/div[3]/div/div[4]/div[2]/form/div[1]/input').send_keys('12345678901')
time.sleep(2)
el2=web.find_element(By.XPATH,'/html/body/div[3]/div/div[4]/div[2]/form/div[1]/div[3]').click()
time.sleep(1)

将验证码截图，不需要保留提示和题目要求。然后用超级鹰解析截图。

In [12]:
from chaojiying import Chaojiying_Client
chaojiying=Chaojiying_Client('YOUR_ACCOUNT','YOUR_PASSWORD','YOUR_ID')
img_div=web.find_element(By.XPATH,'/html/body/div[4]/div[2]/div[6]/div/div/div[2]/div[1]/div/div[2]/img')
img_div.screenshot('1.png')#获取验证码图片
img=img_div.screenshot_as_png
location_click=chaojiying.PostPic(img,'9004')
location_click['pic_str']

'189,117|65,101'

接下来需要让鼠标移动到某一个位置进行点击，这就要用到selenium中的模块：
```
from selenium.webdriver.common.action_chains import ActionChains
```
这被称作“事件链”，所谓事件链，就是一系列行为的序列。例如：

鼠标移动到位置A $\rightarrow$ 点击位置A $\rightarrow$ 鼠标移动到位置B $\rightarrow$ 点击位置B $\rightarrow \cdots$ 

在B站，图片的原点是中心位置，所以，经过不断的尝试，最后得到了：超级鹰得到的x,y都应该减小135.这个超级鹰识别的精确度还是可以的。下面执行点击动作，然后点击确定按钮一次。

In [13]:
from selenium.webdriver.common.action_chains import ActionChains#事件链
# 如同： 鼠标移动到位置
location_list=location_click['pic_str'].split('|')
for location in location_list:
    x,y=location.split(',')
    x,y=int(x)-135,int(y)-135
    # print(x,y)
    #让鼠标移动到(x,y)处，然后点击。
    ac=ActionChains(web)#绑定事件链到浏览器。
    ac.move_to_element_with_offset(img_div,x,y).click().perform()
    time.sleep(1)
    #带着偏移量地将鼠标在网页元素区域内部进行移动(把网页元素的一端视作原点，然后利用偏移量offset来移动鼠标)。
    #所以这里利用刚才抓到手的网页元素img_div，进行偏移量移动鼠标
    #这里一行就是一个事件链，最终必须要perform,确认执行。否则不会执行。
time.sleep(0.5)
comfirmbtn=web.find_element(By.XPATH,'/html/body/div[4]/div[2]/div[6]/div/div/div[3]/a').click()

### 如果你使用驱动程序被浏览器发现了应该怎么办？
被发现的原理是：在开发者界面的console里输入表达式`window.navigator.webdriver`,如果返回值是true,说明你在使用驱动工具操纵浏览器，应该予以防御；如果是用户操纵浏览器，这个选择应该是false.
1. 如果chrome的版本小于88：
在你启动浏览器(web.get)之前，向页面嵌入下面这一段javascript代码，去掉webdriver.
```
[web=Chrome()]
web.execute_cdp_cmd("Page.addScriptToEvaluateOnNewDocument",{
    "source":"""
    navigator.webdriver=undefined
        Object.defineProperty(navigator,'webdriver',{
            get:()=>undefined
        })
    """
})
[web.get(...)]

```

2. 如果chrome的版本大于等于88：引入Options模块，然后添加下面这一段代码。

In [15]:
from selenium.webdriver.chrome.options import Options
option=Options()
option.add_experimental_option('excludeSwitches',['enable-automation'])#这句可加可不加
option.add_argument('--disable-blink-features=AutomationControlled')
web=Chrome()
web.get('https://www.bilibili.com/')


这样执行完毕之后，整体就变成undefined了。

最后，附上一段鼠标完成拖拽动作的代码，以备不时之需。因为我实在找不到有拖拽条人机验证的网页。

In [ ]:
btn=web.find_element(By.XPATH,'PATH_TO_THE_DRAGGING_BAR')
ActionChains(web).drag_and_drop_by_offset(btn,300,0).perform()#这个300，0可以利用截图工具显示的像素尺寸来近似。横着向右拖拽300，同时纵向向上拖拽0.

### 总结
selenium能够非常有效地获取到绝大多数网站的内容，但是遇见能够准确识别selenium的网址，它也无能为力。不过在其他的任何具有加密流程的网页来看，它的确是最为合适的选项。